# Auto CoT - Auto Chain-of-Thought

É uma extensão automatizada da técnica tradicional de Chain-of-Thought (CoT). A ideia central é gerar automaticamente cadeias de raciocínio passo a passo (em vez de criá-las manualmente) para instruir modelos de linguagem em tarefas de raciocínio complexo.

In [1]:
# Imports necessários
from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown
from pathlib import Path
import os
from dotenv import load_dotenv

In [2]:
dotenv_path = Path("../.env")
load_dotenv(dotenv_path=dotenv_path)

True

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-4o-mini")

## **PROMPT 01:** Aqui vou fazer um prompt Auto CoT para o analista de QA

In [5]:
prompt = """
Você é um engenheiro de QA experiente, responsável por validar funcionalidades de sistemas web.

Sua tarefa é gerar **cadeias de raciocínio passo a passo** para criar planos de teste baseados nos requisitos da tela de login. 
Para cada requisito, pense de **três formas diferentes** sobre como ele pode ser validado, incluindo:
- Objetivo da regra
- Casos de sucesso
- Casos de erro
- Casos limite
- Possíveis falhas de segurança
- Considerações de acessibilidade ou usabilidade

Requisitos:
1. O campo de nome de usuário deve aceitar apenas e-mails válidos.
2. A senha deve ter entre 8 e 20 caracteres, contendo letras maiúsculas, minúsculas, números e caracteres especiais.
3. O sistema deve permitir no máximo 3 tentativas de login. Após isso, um captcha deve ser exibido.
4. O link "lembrar senha" deve redirecionar corretamente para a tela de recuperação.

**Instruções:**
- Para cada requisito, gere **3 raciocínios distintos**, como se estivessem vindo de profissionais diferentes.
- Não gere ainda os testes. Apenas pense passo a passo sobre como cada requisito deve ser abordado.
- Use uma estrutura clara como:
  - Raciocínio A:
  - Raciocínio B:
  - Raciocínio C:

Por fim, **gere os testes com base no melhor raciocínio de cada requisito**, no formato Gherkin (Given–When–Then), conforme exemplo abaixo:

```
**Teste 1**: Verificação de acesso com credenciais válidas  
**Cenário**: O usuário entra com dados corretos

**Dado** que o usuário está na tela de login  
**Quando** ele digita o e-mail e a senha corretos  
**Então** o sistema deve redirecioná-lo para a página inicial
```
  
"""

display(Markdown(f"*Prompt:*"))
display(Markdown(f"----"))
display(Markdown(f"{prompt}"))
display(Markdown(f"----"))

response = llm.invoke(prompt)

display(Markdown(f"*Resposta:*"))
display(Markdown(f"----"))
display(Markdown(f"{response.content}"))
display(Markdown(f"----"))

*Prompt:*

----


Você é um engenheiro de QA experiente, responsável por validar funcionalidades de sistemas web.

Sua tarefa é gerar **cadeias de raciocínio passo a passo** para criar planos de teste baseados nos requisitos da tela de login abaixo. Para cada requisito, pense de **três formas diferentes** sobre como ele pode ser validado, incluindo:
- Objetivo da regra
- Casos de sucesso
- Casos de erro
- Casos limite
- Possíveis falhas de segurança
- Considerações de acessibilidade ou usabilidade

Requisitos:
1. O campo de nome de usuário deve aceitar apenas e-mails válidos.
2. A senha deve ter entre 8 e 20 caracteres, contendo letras maiúsculas, minúsculas, números e caracteres especiais.
3. O sistema deve permitir no máximo 3 tentativas de login. Após isso, um captcha deve ser exibido.
4. O link "lembrar senha" deve redirecionar corretamente para a tela de recuperação.

**Instruções:**
- Para cada requisito, gere **3 raciocínios distintos**, como se estivessem vindo de profissionais diferentes.
- Não gere ainda os testes. Apenas pense passo a passo sobre como cada requisito deve ser abordado.
- Use uma estrutura clara como:
  - Raciocínio A:
  - Raciocínio B:
  - Raciocínio C:

Por fim, **gere os testes com base no melhor raciocínio de cada requisito**, no formato Gherkin (Given–When–Then), conforme exemplo abaixo:

```
**Teste 1**: Verificação de acesso com credenciais válidas  
**Cenário**: O usuário entra com dados corretos

**Dado** que o usuário está na tela de login  
**Quando** ele digita o e-mail e a senha corretos  
**Então** o sistema deve redirecioná-lo para a página inicial
```



----

*Resposta:*

----

### Requisito 1: O campo de nome de usuário deve aceitar apenas e-mails válidos.

**Raciocínio A:**
- Objetivo da regra: Garantir que o campo de nome de usuário só receba entradas de e-mail válidas para evitar falhas de autenticação.
- Casos de sucesso: O usuário inseriu "usuario@exemplo.com" e o sistema aceitou.
- Casos de erro: O usuário inseriu "usuario@exemplo" e o sistema rejeitou a entrada.
- Casos limite: O usuário inseriu "a@b.co" (um e-mail válido, mas minimalista) e o sistema aceitou.
- Possíveis falhas de segurança: Um e-mail como "usuario<script>alert('xss')</script>@exemplo.com" deve ser tratado para evitar ataques de injeção.
- Considerações de acessibilidade ou usabilidade: O sistema deve fornecer mensagens claras de erro e dicas de que tipo de e-mail é aceito.

**Raciocínio B:**
- Objetivo da regra: Assegurar a integridade dos dados de login.
- Casos de sucesso: O uso de um e-mail corretamente formatado (ex.: "usuario@dominio.com").
- Casos de erro: E-mails faltando o domínio ("usuario@") ou com formato incorreto ("usuario@@dominio.com").
- Casos limite: E-mails longos perto do limite de 254 caracteres, como "usuario@umdominiolongo.com.br", devem ser aceitos.
- Possíveis falhas de segurança: O sistema não deve permitir que e-mails que contenham comandos maliciosos sejam aceitos.
- Considerações de acessibilidade ou usabilidade: O campo deve ser indicado como "Nome de usuário (e-mail)" para evitar confusão.

**Raciocínio C:**
- Objetivo da regra: Prevenção de falhas de autenticação com e-mails mal formados.
- Casos de sucesso: Usuários que inserem "nome.sobrenome@dominio.com" são capazes de login.
- Casos de erro: E-mails sem o sinal "@" ou com caracteres inválidos devem ser negados pelo sistema.
- Casos limite: E-mails começando ou terminando com um ponto (".") devem ser rejeitados.
- Possíveis falhas de segurança: Permitir e-mails não verificados pode levar à criação de contas falsas ou phishing.
- Considerações de acessibilidade ou usabilidade: Feedback em tempo real durante a digitação do e-mail pode ajudar os usuários.

### Requisito 2: A senha deve ter entre 8 e 20 caracteres, contendo letras maiúsculas, minúsculas, números e caracteres especiais.

**Raciocínio A:**
- Objetivo da regra: Garantir que as senhas sejam suficientemente complexas para aumentar a segurança.
- Casos de sucesso: Uma senha como "Senha123!" é aceita.
- Casos de erro: Uma senha apenas com letras "abcdefg" é rejeitada.
- Casos limite: Senhas de 8 e 20 caracteres com a combinação correta de tipos de caráter devem ser aceitas.
- Possíveis falhas de segurança: Senhas que só usam números devem ser rejeitadas.
- Considerações de acessibilidade ou usabilidade: O sistema deve ter um indicador visual de força da senha.

**Raciocínio B:**
- Objetivo da regra: Prevenir senhas fracas que possam ser facilmente descobertas.
- Casos de sucesso: Senhas complexas que atendem a todos os critérios são aceitas.
- Casos de erro: Senhas com menos de 8 ou mais de 20 caracteres são rejeitadas.
- Casos limite: Testar senhas exatamente na fronteira de 8 e 20 caracteres, como "Abc1!xyz" e "A1!bcdefghijklmnopqrstu".
- Possíveis falhas de segurança: Aceitar senhas que incluem sequências óbvias como "12345678" deve ser evitado.
- Considerações de acessibilidade ou usabilidade: O campo de senha deve incluir uma sugestão de requisitos para ajudar na criação de senhas.

**Raciocínio C:**
- Objetivo da regra: Assegurar que a complexidade da senha evita acessos não autorizados.
- Casos de sucesso: Senhas que combinam todos os requisitos são aceitas.
- Casos de erro: Senhas muito curtas ou que não incluem todos os tipos de caráter são negadas.
- Casos limite: Senhas em preto e branco (ex.: apenas caracteres minúsculos, sem números e sem especiais) devem ser rejeitadas.
- Possíveis falhas de segurança: O sistema deve ter proteção contra tentativas de quebra de senha (brute-force).
- Considerações de acessibilidade ou usabilidade: Fornecer uma função de "mostrar/ocultar" senha pode ajudar usuários a inserir senhas corretamente.

### Requisito 3: O sistema deve permitir no máximo 3 tentativas de login. Após isso, um captcha deve ser exibido.

**Raciocínio A:**
- Objetivo da regra: Proteger o sistema de tentativas de login maliciosas.
- Casos de sucesso: Um usuário faz 3 tentativas com credenciais inválidas e o captcha é mostrado.
- Casos de erro: Um usuário consegue realizar várias tentativas sem ser interrompido.
- Casos limite: Um usuário faz exatamente 3 tentativas antes do captcha.
- Possíveis falhas de segurança: Um captcha mal implementado pode ser contornado ou não funcionar corretamente.
- Considerações de acessibilidade ou usabilidade: O captcha deve ser acessível a usuários com deficiência.

**Raciocínio B:**
- Objetivo da regra: Limitar tentativas de login para aumentar a segurança.
- Casos de sucesso: Após 3 tentativas erradas, o usuário é solicitado a completar um captcha.
- Casos de erro: O sistema não mostra o captcha após 3 tentativas.
- Casos limite: Verificar o comportamento do sistema após a quarta tentativa.
- Possíveis falhas de segurança: Um usuário pode usar ataques distribuídos para contornar o limite.
- Considerações de acessibilidade ou usabilidade: O captcha deve ter opções alternativas para acessibilidade.

**Raciocínio C:**
- Objetivo da regra: Impedir acesso não autorizado através de múltiplas tentativas.
- Casos de sucesso: O sistema mostra o captcha após a terceira tentativa falha.
- Casos de erro: O sistema exibe o captcha em momentos errados ou sem necessidade.
- Casos limite: Testar o que acontece se o usuário tenta logar logo após resolver o captcha.
- Possíveis falhas de segurança: Um captcha que não é aleatório pode ser explorado.
- Considerações de acessibilidade ou usabilidade: O uso de schematics adequados deve garantir que a validação seja amigável.

### Requisito 4: O link "lembrar senha" deve redirecionar corretamente para a tela de recuperação.

**Raciocínio A:**
- Objetivo da regra: Garantir que usuários que esqueceram suas senhas possam recuperá-las de forma fácil.
- Casos de sucesso: O link leva o usuário à página correta de recuperação.
- Casos de erro: O link não redireciona ou leva a uma página de erro.
- Casos limite: Verificar o comportamento do link em diferentes navegadores e dispositivos.
- Possíveis falhas de segurança: O link não deve expor informações sensíveis.
- Considerações de acessibilidade ou usabilidade: O link deve ter um texto claro e acessível.

**Raciocínio B:**
- Objetivo da regra: Facilitar a recuperação de senha com processos claros.
- Casos de sucesso: O usuário clica no link e é redirecionado corretamente.
- Casos de erro: O link está inativo ou leva para uma página errada.
- Casos limite: Navegação do link sem uma conexão de internet ativa.
- Possíveis falhas de segurança: O link não deve permitir que usuários mal-intencionados acessem informações privadas.
- Considerações de acessibilidade ou usabilidade: O link deve ser distinguido visualmente e fácil de encontrar.

**Raciocínio C:**
- Objetivo da regra: Simplificar a recuperação de senhas esquecidas para usuários.
- Casos de sucesso: O link "lembrar senha" é clicado e leva à tela correta.
- Casos de erro: Um erro de redirecionamento leva a uma tela de login novamente.
- Casos limite: Testar a funcionalidade com usuários que não estão logados.
- Possíveis falhas de segurança: O redirecionamento deve ser seguro e não causar vazamento de informações.
- Considerações de acessibilidade ou usabilidade: O tempo de resposta do link deve ser rápido e consistir com práticas de design acessível.

---

### Testes Gherkin com base no melhor raciocínio de cada requisito

**Teste 1**: Validação do campo de nome de usuário para aceitar apenas e-mails válidos  
**Cenário**: Inserir um e-mail válido  
**Dado** que o usuário na tela de login  
**Quando** ele digita "usuario@exemplo.com"  
**Então** o sistema deve aceitar a entrada  

**Teste 2**: Validação das regras de complexidade da senha  
**Cenário**: Inserir senha válida  
**Dado** que o usuário está na tela de login  
**Quando** ele digita uma senha "Senha123!"  
**Então** o sistema deve aceitar a senha  

**Teste 3**: Limitação de tentativas de login  
**Cenário**: Após três tentativas de login falhadas  
**Dado** que o usuário tentou logar com a senha errada três vezes   
**Quando** ele tenta uma quarta vez  
**Então** o sistema deve apresentar um captcha  

**Teste 4**: Redirecionamento do link "lembrar senha"  
**Cenário**: Clicar no link "lembrar senha"  
**Dado** que o usuário está na tela de login  
**Quando** ele clica no link "lembrar senha"  
**Então** o sistema deve redirecioná-lo para a tela de recuperação de senha  

----